# 데이터 전처리 _ 프로젝트 실습

In [2]:
# 1.
# selloutdata 자료에서 음수는 0, 양수는 기존값을 유지하는 QTY_NEW 컬럼 추가하기

# 라이브러리 선언
import pandas as pd
import numpy as np

# 데이터 불러오기
selloutData = pd.read_csv("../../dataset/kopo_channel_seasonality_new.csv", encoding = "ms949")

# 필터링을 수행할 함수식 작성
def valueCluster(inValue):
    outValue = inValue
    
    if inValue < 0:
        outValue = 0
    else :
        outValue = inValue
    return outValue

# 함수식 적용 및 저장
selloutData["QTY_NEW"] = selloutData.QTY.apply(valueCluster)

# 검산식 (0이 나와야 함)
# selloutData.QTY_NEW.min()

In [6]:
# 2.
# selloutdata 자료에서 year, week 컬럼을 생성하고 week가 52 이하인 데이터만 조회
# refinedSelloutData 변수에 담기

# 데이터 형변환
yearweekStr = selloutData.astype({"YEARWEEK" : str})

# year, week 컬럼 생성
selloutData["YEAR"] = yearweekStr.YEARWEEK.str[0:4]
selloutData["WEEK"] = yearweekStr.YEARWEEK.str[4:6]

# 추출 목표 설정
targetweeks = "53"

# 필요내용 포함한 변수 지정
refinedSelloutData = selloutData.loc[selloutData.WEEK <= targetweeks]

# 검산식 (조회내용 없어야 함)
# test = refinedSelloutData.loc[refinedSelloutData.WEEK > targetweeks]

In [7]:
# 3.
# refinedSelloutData에서 지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
# sortedData 변수에 담기

# 정렬 대상 key 변수지정
sortkey = ["REGIONID", "PRODUCT", "YEARWEEK"]

# 정렬 및 변수지정
sortedData = refinedSelloutData.sort_values(by = sortkey, 
                                            ascending = [True, True, True], 
                                            ignore_index=False, inplace = False)

In [8]:
# 4.
# sortedData에서 지역, 상품, 연도 단위의 판매량(QTY_NEW) 평균 연산 후
# groupData 변수에 담고, 이후 컬럼명을 QTY_MEAN으로 변경하기

# 기준이 될 그룹키 지정
groupkey = ["REGIONID", "PRODUCT", "YEAR"]

# 변수 지정 및 평균값 산출
# 그룹키가 인덱스가 되어 정렬하지 못하므로 이를 방지하기 위해 인덱스값을 초기화
# 데이터 특징을 파악할 땐 reset_index() 가 아닌 describe() 가 용이하다.
groupData = sortedData.groupby(by = groupkey)['QTY_NEW'].agg(["mean"]).reset_index()

# 컬럼값 변경
groupData = groupData.rename(columns = {'mean' : 'QTY_MEAN'})

In [9]:
groupData["MA5"] = groupData.QTY_MEAN.rolling(window = 5, 
                                            min_periods = 1, 
                                            center = True).mean()

In [10]:
groupData["EFFECT"] = groupData.QTY_MEAN / groupData.MA5

In [11]:
groupData.YEAR.drop_duplicates()

0    2014
1    2015
2    2016
Name: YEAR, dtype: object

In [12]:
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN,MA5,EFFECT
0,A00,PRODUCT34,2014,275.961538,132.745525,2.078876
1,A00,PRODUCT34,2015,85.698113,100.227413,0.855037
2,A00,PRODUCT34,2016,36.576923,81.691364,0.447745
3,A00,PRODUCT58,2014,2.673077,87.972134,0.030385
4,A00,PRODUCT58,2015,7.547170,5554.570972,0.001359
...,...,...,...,...,...,...
2377,A77,PRODUCT1,2015,3031.547170,22697.924819,0.133561
2378,A77,PRODUCT1,2016,3375.326923,2850.191582,1.184246
2379,A77,PRODUCT12,2014,2035.788462,3365.091582,0.604973
2380,A77,PRODUCT12,2015,3545.641509,3448.477685,1.028176
